In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the path to your dataset
data_dir = 'C:/Users/Badari/OneDrive/Desktop/projects/girls/Cervical Cancer/data'

# Image preprocessing and augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split 20% for validation
)

# Image preprocessing for validation data
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Create generators for training and validation
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    subset='training'  # Set as training data
)

validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    subset='validation'  # Set as validation data
)

# Load the pre-trained MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model layers
base_model.trainable = False

# Create a new model on top of the pre-trained base
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(6, activation='softmax')(x)  # 6 classes including normal

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and model checkpointing
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model with specified parameters
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[model_checkpoint, early_stopping]
)

# Load the saved model for evaluation
saved_model = tf.keras.models.load_model('best_model.keras')

# Evaluate the saved model on the validation set
loss, accuracy = saved_model.evaluate(validation_generator)
print(f'Saved Model Validation Accuracy: {accuracy * 100:.2f}%')


Found 24000 images belonging to 6 classes.
Found 6000 images belonging to 6 classes.
Epoch 1/50


C:\Users\Badari\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


750/750 ━━━━━━━━━━━━━━━━━━━━ 552s 724ms/step - accuracy: 0.7212 - loss: 1.2307 - val_accuracy: 0.9297 - val_loss: 0.1854
Epoch 2/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 389s 518ms/step - accuracy: 0.8520 - loss: 0.3987 - val_accuracy: 0.9263 - val_loss: 0.2038
Epoch 3/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 379s 504ms/step - accuracy: 0.8762 - loss: 0.3398 - val_accuracy: 0.9518 - val_loss: 0.1356
Epoch 4/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 360s 479ms/step - accuracy: 0.8898 - loss: 0.2883 - val_accuracy: 0.9612 - val_loss: 0.1045
Epoch 5/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 400s 532ms/step - accuracy: 0.9037 - loss: 0.2645 - val_accuracy: 0.9642 - val_loss: 0.1030
Epoch 6/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 402s 534ms/step - accuracy: 0.9139 - loss: 0.2317 - val_accuracy: 0.9662 - val_loss: 0.0933
Epoch 7/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 423s 562ms/step - accuracy: 0.9154 - loss: 0.2419 - val_accuracy: 0.9800 - val_loss: 0.0647
Epoch 8/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 430s 571ms/step - accuracy: 0.9265 - loss: 0.20

C:\Users\Badari\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 322 variables whereas the saved optimizer has 10 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


188/188 ━━━━━━━━━━━━━━━━━━━━ 58s 296ms/step - accuracy: 0.9793 - loss: 0.0680
Saved Model Validation Accuracy: 98.00%
